In [2]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
ingest_create_bankchurn_athena_db_passed = False

In [5]:
%store -r ingest_create_bankchurn_athena_db_passed

In [6]:
try:
    ingest_create_bankchurn_athena_db_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [7]:
print(ingest_create_bankchurn_athena_db_passed)

True


In [8]:
if not ingest_create_bankchurn_athena_db_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [9]:
%store -r s3_private_path_bankchurn_csv

In [10]:
try:
    s3_private_path_bankchurn_csv
except NameError:
    print("*****************************************************************************")
    print("[ERROR] PLEASE RE-RUN THE PREVIOUS COPY TSV TO S3 NOTEBOOK ******************")
    print("[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************")
    print("*****************************************************************************")

In [11]:
print(s3_private_path_bankchurn_csv)

s3://sagemaker-us-east-1-994366592132/aai540-finalproject/Churn_Modelling.xls


In [12]:
from pyathena import connect

In [13]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [14]:
# Set Athena parameters
database_name = "bankchurn"
table_name_csv = "churn_modelling"

In [15]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [16]:
# Define the columns and their data types for your CSV file
csv_columns = """
    RowNumber int,
    CustomerID int,
    Surname string,
    CreditScore int,
    Geography string,
    Gender string,
    Age int,
    Tenure int,
    Balance float,
    NumOfProducts int,
    HasCrCard int,
    IsActiveMember int,
    EstimatedSalary float,
    Exited int,
    CreditPerAge float,
    BalanceToSalaryRatio float
"""

# SQL statement to execute with skip.header.line.count property
statement = f"""CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name_csv}(
    {csv_columns}
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION 's3://sagemaker-us-east-1-994366592132/aai540-finalproject/'
TBLPROPERTIES ('skip.header.line.count'='1')
"""

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS bankchurn.churn_modelling(
    
    RowNumber int,
    CustomerID int,
    Surname string,
    CreditScore int,
    Geography string,
    Gender string,
    Age int,
    Tenure int,
    Balance float,
    NumOfProducts int,
    HasCrCard int,
    IsActiveMember int,
    EstimatedSalary float,
    Exited int,
    CreditPerAge float,
    BalanceToSalaryRatio float

) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-994366592132/aai540-finalproject/'
TBLPROPERTIES ('skip.header.line.count'='1')



In [17]:
import pandas as pd

pd.read_sql(statement, conn)

/tmp/ipykernel_200/3803073958.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [18]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_200/2201015668.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,churn_modelling


In [20]:
if table_name_csv in df_show.values:
    ingest_create_bankchurn_athena_table_csv_passed = True

In [21]:
%store ingest_create_bankchurn_athena_table_csv_passed

Stored 'ingest_create_bankchurn_athena_table_csv_passed' (bool)


In [30]:
# Example query to fetch rows where track name doesn't exist in the dataset
search_surname = "Onio"
query_search_surname = f"""SELECT * FROM {database_name}.{table_name_csv}
                        WHERE Surname = '{search_Custid}'
                        """

print(query_search_surname)

SELECT * FROM bankchurn.churn_modelling
                        WHERE Surname = '15634602'
                        


In [31]:
df = pd.read_sql(query_search_surname, conn)
df.head(5)

/tmp/ipykernel_200/1347611181.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_search_surname, conn)


,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited,creditperage,balancetosalaryratio


In [32]:
if not df.empty:
    print("[OK]")
else:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")

++++++++++++++++++++++++++++++++++++++++++++++++++++++
[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.
++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [33]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#">AWS Glue Catalog</a></b>'.format(
            region
        )
    )
)

/tmp/ipykernel_200/4130537117.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [34]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>